In [10]:
# 모듈 임포트
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib
import time
import matplotlib.pyplot as plt
import glob
import os


In [ ]:
# 컬럼명 정의
column_names = ['timestamp', 'v_raw', 'c_raw', 'voltage', 'current', 'label']

# 정상 데이터 불러오기
normal_dir = '../data/normal/'
normal_files = glob.glob(os.path.join(normal_dir, '*.csv'))
normal_dfs = [pd.read_csv(file, names=column_names, header=None) for file in normal_files]
normal_data = pd.concat(normal_dfs, ignore_index=True)

# 아크 데이터 불러오기 (이 부분 주석 해제 필요)
arc_dir = '../data/arc/'
arc_files = glob.glob(os.path.join(arc_dir, '*.csv'))
arc_dfs = [pd.read_csv(file, names=column_names, header=None) for file in arc_files]
arc_data = pd.concat(arc_dfs, ignore_index=True)

# 전체 병합
train_df = pd.concat([normal_data, arc_data], ignore_index=True)

# 셔플
train_df = train_df.sample(frac=1).reset_index(drop=True)

In [12]:
# 파생 피처 추가 함수
def add_features(df):
    df['voltage_diff'] = df['voltage'].diff().fillna(0).abs()
    df['current_diff'] = df['current'].diff().fillna(0).abs()
    df['voltage_ma'] = df['voltage'].rolling(5).mean().bfill()
    df['current_ma'] = df['current'].rolling(5).mean().bfill()
    df['power'] = df['voltage'] * df['current']
    df['power_diff'] = df['power'].diff().fillna(0).abs()
    return df

train_df = add_features(train_df).dropna()


In [13]:
features = ['voltage', 'current', 'voltage_diff', 'current_diff',
            'voltage_ma', 'current_ma', 'power', 'power_diff']
X_train = train_df[features].values
y_train = train_df['label'].values.astype(int)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)


In [14]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# 저장
joblib.dump(model, './model/random_forest_model.joblib')
joblib.dump(scaler, './model/scaler.joblib')

['./model/scaler.joblib']